In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# Remove duplicate theo cột Location và genericCode
deployment_master_df = pd.read_csv('new_deployment_master.csv')
duplicates_deployment_master = deployment_master_df.drop_duplicates(subset=['Location Code','Generic Code'], keep='last')
duplicates_deployment_master = duplicates_deployment_master.sort_values(by=['Generic Code','Location Code'] )
duplicates_deployment_master


FileNotFoundError: [Errno 2] No such file or directory: '/home/minh/Downloads/new_deployment_master.csv'

In [ ]:
# nhặt các giá trị genericCode unique theo Location vào bảng genericCode_ROSn

genericCode_ROSn = pd.DataFrame()
genericCode_ROSn['Location Code'] = duplicates_deployment_master['Location Code']
genericCode_ROSn['Generic Code'] = duplicates_deployment_master['Generic Code']
genericCode_ROSn = genericCode_ROSn.reset_index(drop=True)
genericCode_ROSn


In [ ]:
# /home/minh/python_scripts/python-scripts/public/csv

# Từ giá trị Generic code, lookup trong file master data ra giá trị Brand, sub brand, category,
# sau đó look up các giá trị đó tìm ra giá trị của cột Min Pivotal Size trong file Pivotal Size

master_data_df = pd.read_csv('/home/minh/python_scripts/python-scripts/public/csv/clean_data_masterData.csv')


# đổi tên column
master_data_df.columns = master_data_df.columns.str.replace('Cross-plant CM', 'Generic Code')


# gộp bảng genericCode_ROSn vào bảng  master_data_df
master_data_df.merge(genericCode_ROSn, how='inner', on='Generic Code')

# lọc giá trị dựa trên Generic Code xóa trùng lặp  Generic Code vì có nhiều mặt hàng
Generic_Code = master_data_df.drop_duplicates(subset=['Generic Code'], keep='last')

# gộp bảng Generic_Code vào bảng  genericCode_ROSn 
genericCode_ROSn_merge =genericCode_ROSn.merge(Generic_Code, how='inner', on='Generic Code')

genericCode_ROSn['Brand'] = genericCode_ROSn_merge['Brand']
genericCode_ROSn['Sub-Brand'] = genericCode_ROSn_merge['Sub-Brand']
genericCode_ROSn['Category'] = genericCode_ROSn_merge['Category Type']

genericCode_ROSn
# master_data_df
# genericCode_ROSn = genericCode_ROSn.replace(to_replace='', value='T', regex=True)


In [ ]:
# mở file PivotalSizeMaster để tìm  Min. Pivotal Sizes for ROSN
pivotal_size_master_df = pd.read_csv('/home/minh/python_scripts/python-scripts/public/csv/PivotalSizeMaster.csv')

# đổi tên cho trùng nhau
pivotal_size_master_df.columns = pivotal_size_master_df.columns.str.replace('Store', 'Location Code')

# vì dữ liệu 2 bảng k trùng nên phải replace dữ liệu cho trùng
pivotal_size_master_df = pivotal_size_master_df.replace(to_replace='Arrow', value='ARR', regex=True)
pivotal_size_master_df = pivotal_size_master_df.replace(to_replace='USPA', value='USP', regex=True)


# gộp bảng lấy ra dữ liệu pivotal_size_master_merge  có trùng 3 'Brand','Sub-Brand','Category' 
pivotal_size_master_merge = genericCode_ROSn.merge(pivotal_size_master_df, how='left', on=['Brand','Sub-Brand','Category'])


# kiểm tra nếu location bằng nhau thì lấy nếu gộp ở trên thì sẽ mất vì k kiểm tra đc Generic Code
pivotal_size_master_merge = pivotal_size_master_merge[pivotal_size_master_merge['Location Code_x'] == pivotal_size_master_merge['Location Code_y']]

# đổi tên về tên của bảng genericCode_ROSn
pivotal_size_master_merge.columns = pivotal_size_master_merge.columns.str.replace('Location Code_x', 'Location Code')

# lọc ra lữ liệu cuối cùng nhưng thằng nào mà k đủ dữ liệu sẽ chuyển về NaN
genericCode_ROSn = genericCode_ROSn.merge(pivotal_size_master_merge, how='left', on=['Location Code','Generic Code'])

# add genericCode_ROSn_v1
genericCode_ROSn_v1 = pd.DataFrame()
genericCode_ROSn_v1['Location Code'] = genericCode_ROSn['Location Code']
genericCode_ROSn_v1['Generic Code'] = genericCode_ROSn['Generic Code']
genericCode_ROSn_v1['Min. Pivotal Sizes for ROSN'] = genericCode_ROSn['Min. Pivotal Sizes for ROSN']
genericCode_ROSn_v1

# export excel
genericCode_ROSn_v1.to_csv('genericCode_ROSn_v1.csv',index=False)